# Approach 2: Neural Network on Hand Landmarks

**Data:** 63 features (21 landmarks × 3 coords) → 29 classes

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns

print("TensorFlow version:", tf.__version__)

In [ ]:
# Load and prepare data
df = pd.read_csv('../data/asl_landmarks_train.csv')
df_clean = df.dropna()

X = df_clean.drop('label', axis=1).values
y = df_clean['label'].values

print(f"Samples: {len(X)} | Features: {X.shape[1]} | Classes: {len(np.unique(y))}")

In [ ]:
# Encode labels & split data
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

In [ ]:
# Normalize features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Build neural network
num_classes = len(label_encoder.classes_)

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(63,)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

In [ ]:
# Plot training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history.history['accuracy'], label='Train')
ax1.plot(history.history['val_accuracy'], label='Val')
ax1.set_title('Accuracy'); ax1.legend()
ax2.plot(history.history['loss'], label='Train')
ax2.plot(history.history['val_loss'], label='Val')
ax2.set_title('Loss'); ax2.legend()
plt.show()

In [ ]:
# Evaluate
_, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

In [ ]:
# Classification report
y_pred = model.predict(X_test_scaled).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# Confusion matrix
plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted'); plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Save model
import joblib
model.save('../data/nn_landmark_model.keras')
joblib.dump(label_encoder, '../data/label_encoder.joblib')
joblib.dump(scaler, '../data/scaler.joblib')
print("Model saved!")